In [1]:
import attr
import numpy as np
from thermo import Chemical
import warnings

from gibbs.models.ceos import PengRobinson78
from gibbs.mixture import Mixture
from gibbs.stability_analysis import stability_test

warnings.filterwarnings('ignore')

## Case 1

In [2]:
@attr.s(auto_attribs=True)
class NichitaModel:
    z: np.ndarray
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray
        
    @property
    def mixture(self):
        return Mixture(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor
        )

    @property
    def model(self):
        return PengRobinson78(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

In [3]:
methane = Chemical('methane')
ethane = Chemical('ethane')
nitrogen = Chemical('nitrogen')

* Feed composition 1:

In [4]:
z = np.array([0.10, 0.60, 0.30])
omegas = np.array([methane.omega, ethane.omega, nitrogen.omega])
Tcs = np.array([methane.Tc, ethane.Tc, nitrogen.Tc])
Pcs = np.array([methane.Pc, ethane.Pc, nitrogen.Pc])
kijs = np.array([
    [0.000, 0.021, 0.038],
    [0.021, 0.000, 0.080],
    [0.038, 0.080, 0.000]
])
model = NichitaModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [5]:
P = 7.6e6
T = 270

result = stability_test(model, P, T, z)

result.phase_split

True

* Feed composition 2:

In [6]:
z = np.array([0.30, 0.55, 0.15])
model = NichitaModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [7]:
result = stability_test(model, P, T, z)

result.phase_split

True

* Feed composition 3:

In [8]:
z = np.array([0.38, 0.54, 0.08])
model = NichitaModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [9]:
result = stability_test(model, P, T, z)

result.phase_split

False

* Feed composition 4:

In [10]:
z = np.array([0.05, 0.90, 0.05])
model = NichitaModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [11]:
result = stability_test(model, P, T, z)

result.phase_split

False

## Case 2: Yarborough 8 mixture

In [12]:
propane = Chemical('propane')
pentane = Chemical('pentane')
heptane = Chemical('heptane')
decane = Chemical('decane')

In [13]:
z = np.array([0.8097, 0.0566, 0.0306, 0.0457, 0.0330, 0.0244])
omegas = np.array([
    methane.omega, 
    ethane.omega, 
    propane.omega,
    pentane.omega,
    heptane.omega,
    decane.omega
])
Tcs = np.array([
    methane.Tc, 
    ethane.Tc, 
    propane.Tc,
    pentane.Tc,
    heptane.Tc,
    decane.Tc
])
Pcs = np.array([
    methane.Pc, 
    ethane.Pc, 
    propane.Pc,
    pentane.Pc,
    heptane.Pc,
    decane.Pc
])
kijs = np.zeros((len(Pcs), len(Pcs)))
model = NichitaModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

* Condition 1:

In [14]:
P = 21000000  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split

False

* Condition 2:

In [15]:
P = 21066000  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split

False

* Condition 3:

In [16]:
P = 21067000  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split

False

* Condition 4:

In [17]:
P = 21500000  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split

False